In [1]:
import numpy as np
import math
from hw6_dataload import LFD_Data
from hw6_nlt import LinRegNLT2

# HW 7
## Validation
**Given :** Linear Regression with nonlinear transformation.


&Phi;(x) = (1, x<sub>1</sub>, x<sub>2</sub>, x<sub>1</sub><sup>, 2</sup>, x<sub>2</sub><sup>2</sup>, x<sub>1</sub>x<sub>2</sub>, |x<sub>1</sub> - x<sub>2</sub>|, |x<sub>1</sub> + x<sub>2</sub>|)


As this transformation is the same as the one from HW6 (as well as the datasets), I will reuse the classes from the last homework.

In [4]:
rwd_train = "hw6_train.dta"
rwd_test = "hw6_test.dta"
l_reg = math.pow(10.0, -3) #actually, doesn't use regularization but just copying over from last hw anyways.

# load data from external files and init
rwd_data = LFD_Data(rwd_train, rwd_test)
rwd_algo = LinRegNLT2(rwd_data.dim, 7, l_reg)

#editing from hw6 to make more flexible
def rwd_print_error(algo,valid_X, valid_Y, out_X, out_Y):
    #ein = algo.calc_error(in_X, in_Y)
    e_valid = algo.calc_error(valid_X, valid_Y)
    eout = algo.calc_error(out_X, out_Y)
    print("E_valid: %f, E_out: %f" % (e_valid, eout))
    
print("=== LinReg with NLT using the first 25 examples for training (last 10 for validation) ===")
my_k = np.arange(3,8) #up through which cols (0-idx to use for training)
#train without regularization
for k in my_k:
    print("k = %d :" % k)
    rwd_algo.set_k(k)
    rwd_algo.train(rwd_data.train_X[:25,:], rwd_data.train_Y[:25])
    rwd_print_error(rwd_algo, rwd_data.train_X[25:,:], rwd_data.train_Y[25:], rwd_data.test_X, rwd_data.test_Y)

print("")
print("=== LinReg with NLT using the last 10 examples for training (first 25 for validation) ===")
my_k = np.arange(3,8) #up through which cols (0-idx to use for training)
#train without regularization
for k in my_k:
    print("k = %d :" % k)
    rwd_algo.set_k(k)
    rwd_algo.train(rwd_data.train_X[25:,:], rwd_data.train_Y[25:])
    rwd_print_error(rwd_algo, rwd_data.train_X[:25,:], rwd_data.train_Y[:25], rwd_data.test_X, rwd_data.test_Y)

=== LinReg with NLT using the first 25 examples for training (last 10 for validation) ===
k = 3 :
E_valid: 0.300000, E_out: 0.420000
k = 4 :
E_valid: 0.500000, E_out: 0.416000
k = 5 :
E_valid: 0.200000, E_out: 0.188000
k = 6 :
E_valid: 0.000000, E_out: 0.084000
k = 7 :
E_valid: 0.100000, E_out: 0.072000

=== LinReg with NLT using the last 10 examples for training (first 25 for validation) ===
k = 3 :
E_valid: 0.280000, E_out: 0.396000
k = 4 :
E_valid: 0.360000, E_out: 0.388000
k = 5 :
E_valid: 0.200000, E_out: 0.284000
k = 6 :
E_valid: 0.080000, E_out: 0.192000
k = 7 :
E_valid: 0.120000, E_out: 0.196000


## Validation Bias

**Want :** Expected values of e, e1, and e2 given that e1 and e2 are independent, uniformly-distributed random variables over [0,1] and e = min(e1, e2)

In [7]:
vb_n = 100000 #number of random numbers to generate for each e1 and e2
e1 = np.random.uniform(0,1, vb_n)
e2 = np.random.uniform(0,1, vb_n)
e = np.minimum(e1, e2)

ev_e1 = np.average(e1)
ev_e2 = np.average(e2)
ev_e = np.average(e)

print("Expected values: ")
print("e: %f" % ev_e)
print("e1: %f" % ev_e1)
print("e2: %f" % ev_e2)

Expected values: 
e: 0.333768
e1: 0.501344
e2: 0.499188


## Cross Validation

**Given :**
- data points (x,y): (-1,0), (&rho;, 1), (1,0) &SuchThat; &rho; &ge; 0
- choice between two models: constant {h<sub>0</sub>(x) = b}, linear {h<sub>1</sub>(x) = ax + b}

**Want :** value of &rho; that ties squared error measures for both models using leave-one-out cross-validation

**Note :** N = 3
- E<sub>cv</sub> = (1/N) &sum;(n=1;N){e<sub>n</sub>}
- e<sub>n</sub> = sqrt((h(x) - y)<sup>2</sup>)